In [58]:
import tensorflow as ts
from tensorflow import keras
import numpy as np
import tensorflow.keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, InputLayer
from keras.callbacks import Callback

def custom_loss(y_true, y_pred,t,beta, e, x0, T,gamma):
    """
    Calculates the L2 norm of the difference between y_pred and a constant value, plus
    a scaled sum of y_pred, for each example in the batch. Returns the average L2 norm
    over all examples.

    Args:
    y_true: tensor of true labels
    y_pred: tensor of predicted labels

    Returns:
    A tensor representing the mean L2 norm over all examples.
    """
    
     # calculate the difference y_pred[i] - c * sum(y_pred)
    n = K.int_shape(y_pred)[1]
    hs = T / n
    aux = e*beta*(T-t)*(gamma-beta*e*x0)
    aux2 = hs*K.cumsum(y_pred, axis=1)
    aux3 = hs*(K.sum(aux2, axis=1,keepdims=True)-K.cumsum(aux2, axis=1))

    diff = y_pred-aux+beta*beta*e*e*aux3
    
    # calculate the L2 norm for each example
    norma = K.sqrt(K.sum(K.square(diff), axis=1))
    
    # return the average L2 norm over all training examples
    return K.mean(norma)





class SavePredictions(Callback):
    def __init__(self, x_val):
        self.x_val = x_val
        self.predictions = []
    
    def on_epoch_end(self, epoch, logs={}):
        # Faz as previsões no conjunto de validação
        y_pred = self.model.predict(self.x_val)
        
        # Salva as previsões em uma lista
        self.predictions.append(y_pred)
        
        # Salva as previsões em um arquivo a cada 5 épocas
        if epoch % 10 == 0:
            np.save('solutions/predictions_ex2_epoch{}.npy'.format(10000+epoch), y_pred)



In [59]:
# parâmetros
gamma, beta, e, x0, T = 1.0, 1.0, 1.0, 5.0, 1.0

# Tamanho da entrada, número de neurônios nas camadas ocultas
n, n1, n2 = 100, 5, 5


# Dados de treinamento
X_train = np.linspace(0,T,n).reshape(1,n)
y_train = np.linspace(0,T,n).reshape(1,n)

# Criação do modelo
model = Sequential()

# Adicionando as camadas ao modelo
model.add(InputLayer(input_shape=(n,)))
model.add(Dense(n1, activation='relu'))
model.add(Dense(n2, activation='relu'))
model.add(Dense(n, activation='linear'))

# Compilando o modelo
loss = lambda y_true, y_pred: custom_loss(y_true, y_pred, np.linspace(0,T,n), beta, e, x0, T,gamma)
model.compile(loss=loss, optimizer='adam')


# Treinando o modelo
save_predictions = SavePredictions(X_train)
model.fit(X_train, y_train, epochs=1001, callbacks=[save_predictions])

# model.fit(X_train, y_train, epochs=2001)

# Fazendo uma previsão
y_pred = model.predict(X_train).ravel()

Epoch 1/1001
1/1 [==============================] - 1s 1s/step - loss: 23.1670
Epoch 2/1001
1/1 [==============================] - 0s 77ms/step - loss: 23.1488
Epoch 3/1001
1/1 [==============================] - 0s 75ms/step - loss: 23.1315
Epoch 4/1001
1/1 [==============================] - 0s 75ms/step - loss: 23.1151
Epoch 5/1001
1/1 [==============================] - 0s 70ms/step - loss: 23.1027
Epoch 6/1001
1/1 [==============================] - 0s 72ms/step - loss: 23.0903
Epoch 7/1001
1/1 [==============================] - 0s 74ms/step - loss: 23.0778
Epoch 8/1001
1/1 [==============================] - 0s 70ms/step - loss: 23.0654
Epoch 9/1001
1/1 [==============================] - 0s 81ms/step - loss: 23.0529
Epoch 10/1001
1/1 [==============================] - 0s 110ms/step - loss: 23.0403
Epoch 11/1001
1/1 [==============================] - 0s 103ms/step - loss: 23.0278
Epoch 12/1001
1/1 [==============================] - 0s 137ms/step - loss: 23.0152
Epoch 13/1001
1/1 [=====

In [66]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import io

# Define o número de iterações
directory = os.getcwd()
extensao = ".npy"
files = os.listdir(os.path.join(directory, "solutions"))

filtered_files = sorted([file for file in files if file.endswith(extensao) and "_ex2_" in file])
num_iter = len(filtered_files)

# Cria uma lista para armazenar as imagens em memória
images = []

# Loop pelas iterações
for i in range(num_iter):
    data = np.load("solutions/"+filtered_files[i])
    # Cria os dados do gráfico
    x = np.linspace(0, 1,n)
    y = data.ravel()
    
    # Cria a figura e o eixo
    fig, ax = plt.subplots()
    sns.lineplot(x=x, y=y, ax=ax)
    ax.set(xlim=[0,1],ylim=[-3,1])
    
    # Adiciona o número da iteração no canto superior direito
    ax.text(0.95, 0.95, f'Iteração: {i}', transform=ax.transAxes, ha='right', va='top')
    
    # Salva a figura em memória como uma imagem
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png', dpi=300)
    buffer.seek(0)
    image = Image.open(buffer)
    images.append(image)
    
    # Fecha a figura
    plt.close()

# Salva o GIF a partir das imagens em memória
images[0].save('animacao_ex2.gif',
               save_all=True,
               append_images=images[1:],
               duration=100,
               loop=0)

# Apaga os arquivos numpy
# for file in filtered_files:
#     os.unlink(file)

In [65]:
filtered_files

['predictions_ex2_epoch10000.npy',
 'predictions_ex2_epoch10010.npy',
 'predictions_ex2_epoch10020.npy',
 'predictions_ex2_epoch10030.npy',
 'predictions_ex2_epoch10040.npy',
 'predictions_ex2_epoch10050.npy',
 'predictions_ex2_epoch10060.npy',
 'predictions_ex2_epoch10070.npy',
 'predictions_ex2_epoch10080.npy',
 'predictions_ex2_epoch10090.npy',
 'predictions_ex2_epoch10100.npy',
 'predictions_ex2_epoch10110.npy',
 'predictions_ex2_epoch10120.npy',
 'predictions_ex2_epoch10130.npy',
 'predictions_ex2_epoch10140.npy',
 'predictions_ex2_epoch10150.npy',
 'predictions_ex2_epoch10160.npy',
 'predictions_ex2_epoch10170.npy',
 'predictions_ex2_epoch10180.npy',
 'predictions_ex2_epoch10190.npy',
 'predictions_ex2_epoch10200.npy',
 'predictions_ex2_epoch10210.npy',
 'predictions_ex2_epoch10220.npy',
 'predictions_ex2_epoch10230.npy',
 'predictions_ex2_epoch10240.npy',
 'predictions_ex2_epoch10250.npy',
 'predictions_ex2_epoch10260.npy',
 'predictions_ex2_epoch10270.npy',
 'predictions_ex2_ep